# Week-3(part-1)

In [2]:
!conda install -c conda-forge geopy --yes #we have to first install gropy module

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         395 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

In [3]:
 !pip install beautifulsoup4 #we have to also install beautifulsoup

     |████████████████████████████████| 122kB 7.2MB/s eta 0:00:01


### 1.Importing libraries

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2.Scrap data from Wikipedia page into a DataFrame

In [5]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [6]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [7]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

#### Using BeautifulSoup

In [8]:
# find the table
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [10]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text)

In [11]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [78]:
#To remove("\n") from each row

toronto_df['PostalCode'] = toronto_df['PostalCode'].replace('\n','',regex=True)
toronto_df['Borough'] = toronto_df['Borough'].replace('\n','',regex=True)
toronto_df['Neighborhood'] = toronto_df['Neighborhood'].replace('\n','',regex=True)
toronto_df.head()



,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### 3.Drop cells with a borough that is "Not assigned"

In [86]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### 4.Group neighborhoods in the same borough

In [87]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(lambda x: ", ".join(x)).to_frame()
#toronto_df_grouped.drop(["PostalCode","Borough","Neighborhood"],axis=1,inplace=True)
toronto_df_grouped.reset_index(inplace=True)
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


### 5.For Neighborhood="Not assigned",the value will be same as Borough

In [88]:
# for Neighborhood="Not assigned", make the value same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
       

### 6.Checking whether it is same as required by the question

In [89]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(dfNew[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M8W,Etobicoke,"Alderwood, Long Branch, Alderwood, Long Branch"
1,M6J,West Toronto,"Little Portugal, Trinity, Little Portugal, Tri..."
2,M4B,East York,"Parkview Hill, Woodbine Gardens, Parkview Hill..."
3,M6S,West Toronto,"Runnymede, Swansea, Runnymede, Swansea"
4,M2J,North York,"Fairview, Henry Farm, Oriole, Fairview, Henry ..."
5,M4H,East York,"Thorncliffe Park, Thorncliffe Park"
6,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
7,M5B,Downtown Toronto,"Garden District, Ryerson, Garden District, Rye..."
8,M1X,Scarborough,"Upper Rouge, Upper Rouge"
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel, Commerce Court..."


### 7.Finally, print the number of rows of the cleaned dataframe

In [91]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)